In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import image_dataset_from_directory
import tkinter as tk
from tkinter import filedialog

In [ ]:
train_dir = 'train'
train_dataset = image_dataset_from_directory(
    train_dir,
)
classes = train_dataset.class_names

In [ ]:
filename = "from_camera.jpg"
modelname = "first_model.keras"

def capture_image():
    cam = cv2.VideoCapture(0)
    cv2.namedWindow("Dog Breed Prediction")

    while True:
        ret, frame = cam.read()
        if not ret:
            print("Failed to grab frame")
            break

        cv2.imshow("Dog Breed Prediction", frame)

        if cv2.waitKey(1) & 0xFF == 13:
            cv2.imwrite(filename, frame)
            print("Image captured!")
            break

    cam.release()
    cv2.destroyAllWindows()

def preprocess_image(img_path, img_size):
    img = image.load_img(img_path, target_size=img_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0
    return img_array

def predict_breed(model, img_array):
    predictions = model.predict(img_array)
    predicted_breed = np.argmax(predictions, axis=1)
    return predicted_breed

def use_existing_image():
    global filename
    filename = filedialog.askopenfilename(title="Select an Image", filetypes=[("Image Files", "*.jpg;*.png")])
    if not filename:
        print("No file selected")
        return False
    return True

def center_window(window, width, height):
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()

    x = (screen_width // 2) - (width // 2)
    y = (screen_height // 2) - (height // 2)

    window.geometry(f'{width}x{height}+{x}+{y}')

def main():
    def on_button_click(choice):
        if choice == "capture":
            capture_image()
        elif choice == "existing" and use_existing_image():
            pass
        else:
            return
        
        window.destroy()
        
        img_size = (224, 224)
        img_array = preprocess_image(filename, img_size)
        
        plt.imshow(img_array[0])
        plt.axis("off")
        plt.show()

        model = load_model(modelname)
        
        predicted_breed = predict_breed(model, img_array)
        print(f"Predicted Breed: {classes[predicted_breed[0]]}")

    window = tk.Tk()
    window.title("Dog Breed Prediction")
    window.config(bg="lightblue")
    
    center_window(window, 400, 200)

    label = tk.Label(window, text="Choose an option:", background="lightblue", font=("Consolas", 18))
    label.pack(pady=10)

    capture_button = tk.Button(window, background="#ffa343", text="Capture Image", font=("Consolas", 14), width=20, command=lambda: on_button_click("capture"))
    capture_button.pack(pady=10)

    existing_button = tk.Button(window, background="#50c878", text="Use Existing Image", font=("Consolas", 14), width=20, command=lambda: on_button_click("existing"))
    existing_button.pack(pady=10)

    window.mainloop()

if __name__ == "__main__":
    main()